In [178]:
import numpy as np
import pandas as pd
import time
import awkward as ak
import matplotlib.pyplot as plt
from particle import Particle
from mpl_toolkits.axes_grid.inset_locator import inset_axes

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  import sys


In [79]:
def find_max_pars(df):
    '''From a given DataFrame return the largest number of particles from one event'''
    
    arr = df['pdgf'].to_numpy()
    arr = ak.Array(arr)
    counts = ak.num(arr)
    
    return np.max(counts)

In [80]:
def pad_value(arr, num, val):
    '''Take in a given 2D awkward array and pad it with a value to a specified number'''
    
    awk = ak.Array(arr)
    padded = ak.pad_none(awk, num, axis = 1)
    arr = ak.fill_none(padded, val)
    arr = ak.to_numpy(arr)
    
    return arr

In [213]:
def make_energy_df(df, pars, is_all, is_leading, is_sum, is_cal_eng, num = 52, fill = 0):
    
    # Pad each array
    pdgf = pad_value(df['pdgf'].to_numpy(), num, 0)
    ef = pad_value(df['Ef'].to_numpy(), num, 0)
    
    pxf = pad_value(df['pxf'].to_numpy(), num, 0)
    pyf = pad_value(df['pyf'].to_numpy(), num, 0)
    pzf = pad_value(df['pzf'].to_numpy(), num, 0)
    
    df_new = pd.DataFrame()
    df_new['event'] = df.index
    
    # Go through each particle #
    for par in pars:
        start_time = time.time()
        
        # make arrays of mom, energy where that particle is for each event
        ef_par = np.where(pdgf == par, ef, fill)
        pxf_par = np.where(pdgf == par, pxf, fill)
        pyf_par = np.where(pdgf == par, pyf, fill)
        pzf_par = np.where(pdgf == par, pzf, fill)
        
        ### SORT BASED ON EF ##
        # Index to sort by 
        index = np.argsort(-ef_par)

        # Sort ef, px, py, pz
        ef_par = np.take_along_axis(ef_par, index, axis=1)
        pxf_par = np.take_along_axis(pxf_par, index, axis=1)
        pyf_par = np.take_along_axis(pyf_par, index, axis=1)
        pzf_par = np.take_along_axis(pzf_par, index, axis=1)
        
        ## Remove zero momentum particles ##
        # get rid of zeros in ef (already zero in pxf, pyf, pzf)
        if(fill == 0):
            ef_par = np.where(pxf_par == 0, 0, ef_par)
        else:
            # Remove from all arrays
            ef_par = np.where(pxf_par == 0, fill, ef_par)
            pxf_par = np.where(pxf_par == 0, fill, pxf_par)
            pyf_par = np.where(pxf_par == 0, fill, pyf_par)
            pzf_par = np.where(pxf_par == 0, fill, pzf_par)
            
        ef_par = ak.to_awkward0(ef_par)
        pxf_par = ak.to_awkward0(pxf_par)
        pyf_par = ak.to_awkward0(pyf_par)
        pzf_par = ak.to_awkward0(pzf_par)
    
        if is_all:
            
            ## Remove extra fill values ##
            ef_pars =  ef_par[~(ef_par == fill)]
            pxf_pars = pxf_par[~(pxf_par == fill)]
            pyf_pars = pyf_par[~(pyf_par == fill)]
            pzf_pars = pzf_par[~(pzf_par == fill)]
        
            ## Add arrays to the new dataframe ##
            df_new['Ef ' + str(par)] = ak.to_numpy(ef_pars)
            df_new['pxf ' + str(par)] = ak.to_numpy(pxf_pars)
            df_new['pyf ' + str(par)] = ak.to_numpy(pyf_pars)
            df_new['pzf ' + str(par)] = ak.to_numpy(pzf_pars)
       
        if is_leading:
            ## Add arrays to the new dataframe ##
            df_new['Ef ' + str(par) + ' l'] = ef_par[::, 0]
            df_new['pxf ' + str(par) + ' l'] = pxf_par[::, 0]
            df_new['pyf ' + str(par) + ' l'] = pyf_par[::, 0]
            df_new['pzf ' + str(par) + ' l'] = pzf_par[::, 0] 
        
        if is_sum:
            ## Add arrays to the new dataframe ##
            df_new['Ef ' + str(par) + ' sum'] = np.sum(ak.Array(ef_par), axis = 1)
            df_new['pxf ' + str(par) + ' sum'] = np.sum(ak.Array(pxf_par), axis = 1)
            df_new['pyf ' + str(par) + ' sum'] = np.sum(ak.Array(pyf_par), axis = 1)
            df_new['pzf ' + str(par) + ' sum'] = np.sum(ak.Array(pzf_par), axis = 1) 
        
        if is_cal_eng:
            ## Add arrays to the new dataframe ##
            p = Particle.from_pdgid(par)
            ef_par = ef_par.astype('float')
            ef_par[ef_par == 0] = np.nan
            
            if (par == 2112 or par == 2212 or par == 3112 or par == 3122 or par == 3212 or par == 3222):
                mass = p.mass/1000
            else:
                mass = 0
            df_new['cal ' + str(par)] = np.sum(np.nan_to_num(ak.Array(ef_par) - mass), axis = 1) 
        
        print('Done with: ', str(par), 'time took: ', time.time() - start_time, ' still working ...')
    return df_new

In [ ]:
path = '/Users/laurazichi/Desktop/base_generation_with_weights_9Jun_FSIFix.hdf'
gst_df = pd.read_hdf(path)

In [127]:
lep_df = gst_df[np.sqrt((gst_df['pxl']**2 + gst_df['pyl']**2)) > 0.4]
lep_cut = np.array(lep_df.index)

In [5]:
lep_df = pd.read_hdf("lep_cut_df.hdf")

In [218]:
pars_all = np.array([-321, -311, -211,  -13,  -11,   11,   13,   22,  111,  211,  311,  321, 2112, 2212,
 3112, 3122, 3212,3222])
df_new = make_energy_df(lep_df, pars_all, False, True, True, True)

Done with:  -321 time took:  5.224646329879761  still working ...
Done with:  -311 time took:  5.190582990646362  still working ...
Done with:  -211 time took:  5.10597825050354  still working ...
Done with:  -13 time took:  5.259606122970581  still working ...
Done with:  -11 time took:  5.105317115783691  still working ...
Done with:  11 time took:  5.273281812667847  still working ...
Done with:  13 time took:  5.178674936294556  still working ...
Done with:  22 time took:  5.239135026931763  still working ...
Done with:  111 time took:  5.169209718704224  still working ...
Done with:  211 time took:  5.170384168624878  still working ...
Done with:  311 time took:  5.100643157958984  still working ...
Done with:  321 time took:  5.357055902481079  still working ...
Done with:  2112 time took:  5.567325115203857  still working ...
Done with:  2212 time took:  5.589590072631836  still working ...
Done with:  3112 time took:  5.378298044204712  still working ...
Done with:  3122 time t

In [219]:
df_new.to_hdf('fast_sorted.hdf', 'lep_fast')